In [ ]:
from flask import Flask, render_template, request,jsonify
from flask_cors import CORS,cross_origin
import requests
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

app = Flask(__name__)


@app.route('/', methods=['GET'])  # route to display the home page
@cross_origin()
def homePage():
    return render_template("index.html")


@app.route('/review', methods=['POST', 'GET'])  # route to show the review comments in a web UI
@cross_origin()
def index():
    if request.method == 'POST':
        try:
            searchString = request.form['content'].replace(" ", "")
            url = "https://www.flipkart.com/search?q=" + searchString
            httpObject = urlopen(url)
            webdata = httpObject.read()
            # print(webdata)
            soupdata = soup(webdata, 'html.parser')

            containers = soupdata.find_all('div', {'class': '_2kHMtA'})

            filename = searchString + ".csv"
            fw = open(filename, "wb")
            headers = 'Product_Name,stars_Scrap,Ratings,Reviews,CurrentPrice,MRP_NOW,Processor,RAM,Storage,Storage_SSD,imageURL\n'.encode()
            fw.write(headers)
            reviews = []
            # file=open("laptop1_info.csv","wb") #b is for binary rupees is available like biinanari forma so give 'w'b''
            # file.write('Product_Name,stars_Scrap,Ratings,Reviews,CurrentPrice,MRP_NOW,Processor,RAM,Storage,Storage_SSD,imageURL\n'.encode())
            for container in containers:
                # Scrapping productnames here
                product = container.find('div', {'class': '_4rR01T'})
                productnames = product.text.split('-')[0].strip()
                try:
                    Product_Name = productnames
                except:
                    print(0)

                # Scrapping Stars here
                Stars = container.find('div', {'class': '_3LWZlK'})
                try:
                    stars_Scrap = Stars.text
                    # print(stars_Scrap)
                    # print('\n')
                except:
                    stars_Scrap = 0
                    # print(stars_Scrap)
                    # print('hello')
                # Scrapping Ratings here
                Rating = container.find('span', {'class': '_2_R_DZ'})
                try:
                    # RetRev=re.findall('\d+,?\d*' , '5,433 Ratings & 134 Reviews')
                    # print(RetRev[0],RetRev[1])
                    RetRev = re.findall('\d+,?\d*', Rating.text)
                    Ratings = RetRev[0].replace(',', '')
                    Reviews = RetRev[1].replace(',', '')
                except:
                    Ratings = 0
                    Reviews = 0
                    # Scrapping currentprice mrp
                currentprice = container.find('div', class_="_30jeq3 _1_WHN1")

                CurrentPrice = currentprice.text.replace(',', '').replace('₹',
                                                                          '')  # here ₹4353 it is give str we have int so we have
                # to remove that with the help of replace
                # Scrapping actual mrp price
                MRP = container.find('div', {'class': '_3I9_wc _27UcVY'})
                # print(MRP)
                try:
                    MRP_NOW = MRP.text.replace(',', '').replace('₹',
                                                                '')  # here ₹4353 it is give str we have int so we have
                    # print(MRP_NOW)                                  #to remove that with the help of replace
                except:
                    print(0)
                    # Scrapping info for the particuler product like ram,rom,ssd
                    info = container.findAll('li', class_='rgWa7D')
                    Processor = info[0].text
                    RAM = info[1].text
                    Storage = info[2].text
                    Storage_SSD = info[3].text
                    # Sraping the imageURL`s
                image = container.img
                imageURL = image.get('src')
                mydict = {"productname":Product_Name, "stars":stars_Scrap, "ratings":Ratings, "reviews":Reviews, "currentPrice":CurrentPrice, "new_price":MRP_NOW, "processor":Processor, "ram":RAM, "storage":Storage, "storage_SSD":Storage_SSD, "imageurl":imageURL}
                reviews.append(mydict)
            return render_template('results.html', reviews=reviews[0:(len(reviews) - 1)])
        except Exception as e:
            print('the Exception massege is ', e)
    else:
        return render_template('index.html')


if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
pages_link=soupdata.findAll('a',{'class':'ge-49M'})
domain='https://www.flipkart.com'

domain+pages_link[0].get('href')

In [ ]:
Product, Customer Name, Rating, Heading, Comment \n
"Product": searchString, "Name": name, 
"Rating": rating, "CommentHead":commentHead,
"Comment": custComment